In [1]:
from datasets import load_dataset
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import pipeline
import torch

In [2]:
def load_sst5(split="train", num_samples=5000):
    dataset = load_dataset("SetFit/sst5", "default", split=split)
    dataset = dataset.filter(lambda x: x["label_text"] is not None)
    dataset = dataset.shuffle(seed=42).select(range(min(num_samples, len(dataset))))

    texts = []
    labels = []
    for sample in dataset:
        text = sample["text"].strip().replace("\n", " ")
        label_text = sample["label_text"]
        full_text = f"Text: {text}\nLabel: {label_text}"
        texts.append(full_text)
        labels.append(label_text)
    return texts, labels

In [3]:
train_texts, train_labels = load_sst5()

Repo card metadata block was not found. Setting CardData to empty.


In [4]:
test_texts, test_labels = load_sst5("test", 1000)

Repo card metadata block was not found. Setting CardData to empty.


In [5]:
def organize_SST5_into_df(texts, labels):
    input_data = []
    for each_text in texts:
        sentiment_line = next(line for line in each_text.split('\n') if line.lower().startswith("text"))
        input_data.append(sentiment_line.lower().split('text:')[1].strip())
    df = pd.DataFrame({
        'text': input_data,
        'sentiment': labels
    })
    return df

In [6]:
# df_train_5000 = organize_SST5_into_df(train_texts, train_labels)
df_test_1000 = organize_SST5_into_df(test_texts, test_labels)

In [7]:
df_test_1000

,text,sentiment
0,a 93-minute condensation of a 26-episode tv se...,negative
1,the premise is overshadowed by the uberviolenc...,negative
2,this is a startling film that gives you a fasc...,positive
3,but you 'll definitely want the t-shirt .,neutral
4,i tried to read the time on my watch .,negative
...,...,...
995,focuses on joan 's raging hormones and sledgeh...,very negative
996,polished korean political-action film is just ...,neutral
997,-lrb- fessenden -rrb- is much more into ambigu...,positive
998,lovingly choreographed bloodshed taking place ...,positive


In [8]:
from sentence_transformers import SentenceTransformer

def encode_text(text):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    return model.encode(text)

from sklearn.metrics.pairwise import cosine_similarity

def select_top_k(query_embedding, candidate_embeddings, k):
    similarities = cosine_similarity([query_embedding], candidate_embeddings)[0]
    top_k_indices = np.argsort(similarities)[-k:][::-1]
    return top_k_indices

In [9]:
# tqdm.pandas()
# df_train_5000['embedding'] = df_train_5000['text'].progress_apply(encode_text)

In [10]:
# df_train_5000.to_pickle("train_5000_with_embeddings_SST5.pkl")

In [11]:
df_train_5000_with_embeddings = pd.read_pickle("train_5000_with_embeddings_SST5.pkl")

In [12]:
def zero_shot_cot(input_, model, tokenizer):
    messages = [
        {"role": "user", "content": "What is the sentiment of the input? negative, neutral, positive, very negative, very positive?\nThe response should follow the format: Sentiment:{negative, neutral, positive, very negative, very positive}\nReason:{reason}" + f"\nInput: {input_}" + "\nLet's think step by step."},
    ]
    pipe = pipeline("text-generation", 
                    model=model,
                    tokenizer=tokenizer,
                    pad_token_id=tokenizer.eos_token_id
                   )
    completion = pipe(messages)
    return completion[0]['generated_text'][1]['content']

def few_shot_cot(examples, input_, model, tokenizer):
    messages = [
        {"role": "user", "content": "What is the sentiment of the input? negative, neutral, positive, very negative, very positive?" + f"\n{examples}" +"\nThe response should follow the format: Sentiment:{negative, neutral, positive, very negative, very positive}\nReason:{reason}\nHere is the test data" + f"\nInput: {input_}" + "\nLet's think step by step."},
    ]
    pipe = pipeline("text-generation", 
                    model=model,
                    tokenizer=tokenizer,
                    pad_token_id=tokenizer.eos_token_id
                   )
    completion = pipe(messages)
    return completion[0]['generated_text'][1]['content']

In [13]:
from collections import Counter
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import logging
logging.set_verbosity_error()

def iterative_demonstration_selection(test_sample, train_samples, model_id, k=4, q=2):

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:0")

    train_embeddings = np.stack(train_samples['embedding'].to_numpy())
    all_answers = []
    reasoning_path = zero_shot_cot(test_sample, model, tokenizer)

    for _ in range(q):
        query_embedding = encode_text(reasoning_path)
        selected_indices = select_top_k(query_embedding, train_embeddings, k)
        demonstrations = [train_samples.iloc[i] for i in selected_indices]

        examples_prompt = "\n".join(
            [f"Input: {row['text']}\nSentiment: {row['sentiment']}" for row in demonstrations]
        )

        result = few_shot_cot(examples_prompt, test_sample, model, tokenizer)
        

        try:
            sentiment_line = next(line for line in result.split('\n') if line.lower().startswith("sentiment"))
            reason_line = next(line for line in result.split('\n') if line.lower().startswith("reason"))
        except StopIteration:
            sentiment_line = "Sentiment: unknown"
            reason_line = "Reason: unknown"

        reasoning_path = reason_line
        answer = sentiment_line.split(":", 1)[-1].strip().lower()
        all_answers.append(answer)

    final_answer = Counter(all_answers).most_common(1)[0][0]
    return final_answer

In [14]:
def run_ids_on_test_set(df_test, df_train, model_id, k=4, q=3):
    predictions = []
    reasoning_paths = []

    for i, row in tqdm(df_test.iterrows(), total=len(df_test), desc="Running IDS on test set"):
        test_text = row['text']
        true_label = row['sentiment']
        
        # Run IDS
        try:
            final_answer = iterative_demonstration_selection(test_text, df_train, k=k, q=q, model_id=model_id)
        except Exception as e:
            final_answer = "error"
            print(f"Error on test sample {i}: {e}")

        predictions.append(final_answer)
        reasoning_paths.append(test_text)

    df_test_result = df_test.copy()
    df_test_result['prediction'] = predictions
    df_test_result['input'] = reasoning_paths

    return df_test_result

In [15]:
from sklearn.metrics import accuracy_score

df_test_result = run_ids_on_test_set(df_test_1000, df_train_5000_with_embeddings, k=4, q=3, model_id="meta-llama/Llama-3.2-3B-Instruct")

Running IDS on test set:   0%|          | 0/1000 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running IDS on test set:   0%|          | 1/1000 [00:11<3:07:03, 11.23s/it]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running IDS on test set:   0%|          | 2/1000 [00:22<3:05:58, 11.18s/it]

...

Running IDS on test set: 100%|█████████▉| 998/1000 [2:51:49<00:19,  9.57s/it]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running IDS on test set: 100%|█████████▉| 999/1000 [2:51:59<00:09,  9.63s/it]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running IDS on test set: 100%|██████████| 1000/1000 [2:52:08<00:00, 10.33s/it]


In [16]:
df_test_result

,text,sentiment,prediction,input
0,a 93-minute condensation of a 26-episode tv se...,negative,very negative,a 93-minute condensation of a 26-episode tv se...
1,the premise is overshadowed by the uberviolenc...,negative,very negative,the premise is overshadowed by the uberviolenc...
2,this is a startling film that gives you a fasc...,positive,very negative,this is a startling film that gives you a fasc...
3,but you 'll definitely want the t-shirt .,neutral,neutral,but you 'll definitely want the t-shirt .
4,i tried to read the time on my watch .,negative,neutral,i tried to read the time on my watch .
...,...,...,...,...
995,focuses on joan 's raging hormones and sledgeh...,very negative,very negative,focuses on joan 's raging hormones and sledgeh...
996,polished korean political-action film is just ...,neutral,neutral,polished korean political-action film is just ...
997,-lrb- fessenden -rrb- is much more into ambigu...,positive,neutral,-lrb- fessenden -rrb- is much more into ambigu...
998,lovingly choreographed bloodshed taking place ...,positive,very negative,lovingly choreographed bloodshed taking place ...


In [17]:
accuracy = accuracy_score(df_test_result['sentiment'], df_test_result['prediction'])
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 36.30%


In [18]:
df_test_result[['input','sentiment','prediction']].to_csv("SST5_ids_results_LLAMA.csv", index=False)

In [31]:
df_test_result_gemma = run_ids_on_test_set(df_test_1000, df_train_5000_with_embeddings, k=4, q=3, model_id="google/gemma-2-2b-it")

Running IDS on test set:   0%|          | 0/1000 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running IDS on test set:   0%|          | 1/1000 [00:17<4:56:33, 17.81s/it]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running IDS on test set:   0%|          | 2/1000 [00:29<3:53:05, 14.01s/it]

...

Running IDS on test set: 100%|█████████▉| 998/1000 [3:03:44<00:21, 10.66s/it]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running IDS on test set: 100%|█████████▉| 999/1000 [3:03:54<00:10, 10.44s/it]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running IDS on test set: 100%|██████████| 1000/1000 [3:04:04<00:00, 11.04s/it]


In [32]:
import re

def clean_prediction(pred):
    pred = pred.lower().strip()
    pred = re.sub(r"\*\*(.*?)\*\*", r"\1", pred)
    pred = re.sub(r"[{}]", "", pred)
    pred = re.sub(r"[^\w\s]", "", pred)
    return pred.strip()

In [33]:
df_test_result_gemma['prediction_clean'] = df_test_result_gemma['prediction'].apply(clean_prediction)

In [34]:
df_test_result_gemma

,text,sentiment,prediction,input,prediction_clean
0,a 93-minute condensation of a 26-episode tv se...,negative,**negative**,a 93-minute condensation of a 26-episode tv se...,negative
1,the premise is overshadowed by the uberviolenc...,negative,**negative**,the premise is overshadowed by the uberviolenc...,negative
2,this is a startling film that gives you a fasc...,positive,**positive**,this is a startling film that gives you a fasc...,positive
3,but you 'll definitely want the t-shirt .,neutral,**positive**,but you 'll definitely want the t-shirt .,positive
4,i tried to read the time on my watch .,negative,**neutral**,i tried to read the time on my watch .,neutral
...,...,...,...,...,...
995,focuses on joan 's raging hormones and sledgeh...,very negative,**very negative**,focuses on joan 's raging hormones and sledgeh...,very negative
996,polished korean political-action film is just ...,neutral,**neutral**,polished korean political-action film is just ...,neutral
997,-lrb- fessenden -rrb- is much more into ambigu...,positive,**positive**,-lrb- fessenden -rrb- is much more into ambigu...,positive
998,lovingly choreographed bloodshed taking place ...,positive,**negative**,lovingly choreographed bloodshed taking place ...,negative


In [35]:
accuracy_gemma = accuracy_score(df_test_result_gemma['sentiment'], df_test_result_gemma['prediction_clean'])
print(f"Accuracy: {accuracy_gemma * 100:.2f}%")

Accuracy: 51.30%


In [36]:
df_test_result[['input','sentiment','prediction']].to_csv("SST5_ids_results_gemma.csv", index=False)